In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_validate,KFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')

In [2]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [3]:
set_train = pd.read_csv('data_set.csv')
test_values = pd.read_csv('data_test.csv', index_col='building_id')

In [4]:
columnas_seleccionada = ['age',
                         'geo_level_1_id',
                         'area_percentage',
                         'count_floors_pre_eq',
                         'geo_level_2_id',
                         'height_percentage',
                         'resistance',
                         'roof_type',
                         'ground_floor_type',
                         'other_floor_type',
                         'position_j', 'position_o', 
                         'position_s', 'position_t',
                          'base_condition',
                         ]

In [5]:
columnas_seleccionads = ['age',
                         'geo_level_1_id',
                         'area_percentage',
                         'count_floors_pre_eq',
                         'geo_level_2_id',
                         'height_percentage',
                         'resistance',
                         'roof_type',
                         'ground_floor_type',
                         'other_floor_type',
                         'position_j', 'position_o', 
                         'position_s', 'position_t',
                          'uso_secundario',
                         'prom_heigh_floor',
                         'base_condition',
                         ]

In [6]:
columnas_seleccionadas= ['age', 'geo_level_1_id', 'geo_level_2_id','geo_level_3_id',
'count_floors_pre_eq', 'area_percentage', 'height_percentage','count_families',
'has_secondary_use','has_secondary_use_agriculture', 'has_secondary_use_hotel', 'has_secondary_use_rental',
'has_secondary_use_institution', 'has_secondary_use_school', 'has_secondary_use_industry', 
'has_secondary_use_health_post', 'has_secondary_use_gov_office','has_secondary_use_use_police',
'has_secondary_use_other',

'roof_type_n', 'roof_type_q', 'roof_type_x',
'ground_floor_type_f','ground_floor_type_m',
'ground_floor_type_v','ground_floor_type_x',
'ground_floor_type_z',
'other_floor_type_j','other_floor_type_q',
'other_floor_type_s','other_floor_type_x',
'foundation_type_h', 'foundation_type_i', 
'foundation_type_r', 'foundation_type_u', 
'foundation_type_w', 'land_surface_condition_n', 
'land_surface_condition_o', 'land_surface_condition_t',
'position_j', 'position_o', 
'position_s', 'position_t',
'plan_configuration_d', 'plan_configuration_u', 
'plan_configuration_s', 'plan_configuration_q', 
'plan_configuration_m', 'plan_configuration_c', 
'plan_configuration_a', 'plan_configuration_n', 
'plan_configuration_f', 'plan_configuration_o',
#'base_condition',  'uso_secundario'#, 
#'roof_type', 'ground_floor_type', 'other_floor_type','uso_secundario',
 #                       'prom_heigh_floor',
                       # 'resistance'
]

In [7]:
train, test = train_test_split(set_train, test_size = 0.20, shuffle = False)

In [8]:
train_values_subset = train.iloc[:, 0:-1]
train_labels = train.loc[:, 'damage_grade']

In [9]:
test_values_subset = test.iloc[:, 0:-1]
test_labels = test.loc[:, 'damage_grade']

In [10]:
# A parameter grid for XGBoost
#n_jobs=[-1]

#n_estimators=np.arange(200,440,20)
#max_depth=np.arange(5,15,1)
#Param grid for Xgboost
#param_grid={'n_jobs':n_jobs,
#            'n_estimators':n_estimators,
#            'max_depth':max_depth,

 #          }

In [11]:
#rf=XGBClassifier()
#kf=KFold(n_splits=2,shuffle=True)

In [12]:
#parametros = rs.best_params_
#parametros

In [13]:
#rs=RandomizedSearchCV(rf,param_distributions=param_grid,cv=kf,scoring='f1_micro')

In [14]:
#random_search = RandomizedSearchCV(rf_model, param_distributions=params, n_iter=500, scoring='accuracy', n_jobs=1, cv=skf.split(train_values_subset,train_labels.values.ravel()), random_state=1)

In [15]:
rf_model = XGBClassifier(n_jobs=-1,n_estimators= 300, max_depth= 10,learning_rate= 0.1, eval_metric='mlogloss',col_by_tree=0.9)
#rs = XGBClassifier(n_jobs=-1,n_estimators= 300, max_depth= 10,learning_rate= 0.1, eval_metric='mlogloss',col_by_tree=0.9)

In [16]:
train_values_subset = pd.get_dummies(train_values_subset)

In [17]:
start_time = timer(None)
rf_model.fit(train_values_subset, train_labels.values.ravel())
timer(start_time)

[23:04:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "col_by_tree" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.



 Time taken: 0 hours 5 minutes and 3.28 seconds.


In [18]:
# *----- Submit actual 0.7257
preds = rf_model.predict(test_values_subset)
f1_score(test_labels, preds, average='micro')

0.7444024481494982

In [19]:
#0.7466855969762669 con search sin learning rate
#0.7460853546208166 sacando prom height floor
#0.7461237334970832 agregando volumen
#0.7477740251765427 con resist
#0.7475821307952104 uso sec sin resist
# 0.7372965919557876      c/uso secundario submit: 0.7285
# 0.7466472247270773 .......

# 0.7462196806877495 -------- 0.740

In [20]:
test_values_subset = test_values[columnas_seleccionadas]
test_values_subset = pd.get_dummies(test_values_subset)
test_values_subset

,age,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,area_percentage,height_percentage,count_families,has_secondary_use,has_secondary_use_agriculture,...,plan_configuration_d,plan_configuration_u,plan_configuration_s,plan_configuration_q,plan_configuration_m,plan_configuration_c,plan_configuration_a,plan_configuration_n,plan_configuration_f,plan_configuration_o
building_id,,,,,,,,,,,,,,,,,,,,,
300051,20,17,596,11307,3,7,6,1,0,0,...,0,0,0,0,0,0,0,0,0,1
99355,25,6,141,11987,2,13,5,1,1,1,...,0,0,0,0,0,0,0,0,0,1
890251,5,22,19,10044,2,4,5,1,0,0,...,0,0,0,0,0,0,0,0,0,1
745817,0,26,39,633,1,19,3,2,1,0,...,0,0,0,0,0,0,0,0,0,1
421793,15,17,289,7970,3,8,7,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310028,70,4,605,3623,3,20,6,1,1,1,...,0,0,0,0,0,0,0,0,0,1
663567,25,10,1407,11907,3,6,7,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1049160,50,22,1136,7712,1,3,3,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [21]:
train_values_subset

,age,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,area_percentage,height_percentage,count_families,has_secondary_use,has_secondary_use_agriculture,...,plan_configuration_d,plan_configuration_u,plan_configuration_s,plan_configuration_q,plan_configuration_m,plan_configuration_c,plan_configuration_a,plan_configuration_n,plan_configuration_f,plan_configuration_o
0,30,6,487,12198,2,6,5,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,10,8,900,2812,2,8,7,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,10,21,363,8973,2,5,5,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,10,22,418,10694,2,6,5,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,30,11,131,1488,3,8,9,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208475,0,8,217,10644,3,11,9,1,0,0,...,0,0,0,0,0,0,0,0,0,1
208476,40,6,487,1755,2,7,5,2,0,0,...,0,0,0,0,0,0,0,0,0,1
208477,40,25,854,2844,2,6,5,1,0,0,...,0,0,0,0,0,0,0,0,0,1
208478,30,20,1309,346,2,6,7,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [22]:
predictions = rf_model.predict(test_values_subset)

ValueError: Feature shape mismatch, expected: 64, got 86868

In [ ]:
submission_format = pd.read_csv('submission_format.csv',index_col='building_id')

In [ ]:
my_submission = pd.DataFrame(data=predictions, columns=submission_format.columns,
                            index=submission_format.index)

In [ ]:
my_submission.head()

In [ ]:
my_submission.to_csv('submision.csv')

In [ ]:
!head submision.csv

In [ ]:
rf_model.feature_importances_

In [ ]:
# Gini importance
rf_model.feature_importances_
columnas_seleccionadas = ['age',
                         'geo_level_1_id',
                         'area_percentage',
                         'count_floors_pre_eq',
                         'geo_level_2_id',
                         'height_percentage',
                         'resistance',
                         'roof_type',
                         'ground_floor_type',
                         'other_floor_type',
                    #     'position_j', 'position_o', 
                     #    'position_s', 'position_t',
                         'owner_a', 'owner_r',
                         'owner_v', 'owner_w',
                         'base_condition']

In [ ]:
plt.bar(test_values_subset.columns, rf_model.feature_importances_)
plt.xlabel('Features')
plt.ylabel('Importancia')
plt.xticks(rotation=90)
plt.title('Importancia Features con RF')
plt.rcParams["figure.figsize"] = (30,8)
plt.show()